## Introduction/ Business Problem

The owner of a Chinese restaurant chain is interested to open their first restaurant in Manhattan, New York. 
For this purpose he is seeking advice on locations of existing similar restaurants. 
In particular, he mentioned that he would like to know the top 10 location/ neighborhood where 
there are a number of Chinese (or similar) restaurant already existing. 
This will help him to further strategize on the location aspect.

Hence here,

### Target audience
The owner of the Chinese restaurant chain 
### Problem statement
To advise on current locations of Chinese (or similar) restaurants in Manhattan



## Data Set to Address the Problem

First we'll utilize the New York location data as available on the web (also in labs as part of the Capstone course). 
It contains the borough and neighborhood information of NY. 
This data should be first extracted into suitable DataFrames - containing Borough, Neighborhood, Latitude, Longitude.

After that we'll focus on the data for Manhattan borough and neighborhoods within this borough. 
The relevant latitude and longitude data will help us to utilize Foursquare APIs. 
Foursquare API will provide us top N number of venues and their categories, latitude, longitude etc. 
Then we'll focus on categories and try to find out number of Chinese (or similar) restaurants in each neighborhoods. 
Additionally, we'll map the top 10 neighborhoods on Manhattan map for easy visualization.


## Data Analysis


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<a id='item1'></a>

## Download and Explore Dataset

The NY data set is available in the internet. However as noted through a previous lab, this is also available through the 
following wget command. So we are utilizing the same for sake of simplicity.


In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

All the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
type(neighborhoods_data)

list

Let's take a look at the first item in this list.

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

We'll transform this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Then let's loop through the data and fill the dataframe one row at a time.

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']  
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## Manhattan Data
Lets now focus on neighborhoods in Manhattan

In [10]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods in Manhattan

#### Define Foursquare Credentials and Version

In [11]:
# @

CLIENT_ID = 'DHDIEPGCPGDONTZLWNZ5DY5GVQXACPZQDD01ZLVT2UN4ZQ0D' # your Foursquare ID
CLIENT_SECRET = '1LLXWMC3KPTSXOQHYD0XV2EYJFY1IC542RW4I1SSU1S1TUNH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DHDIEPGCPGDONTZLWNZ5DY5GVQXACPZQDD01ZLVT2UN4ZQ0D
CLIENT_SECRET:1LLXWMC3KPTSXOQHYD0XV2EYJFY1IC542RW4I1SSU1S1TUNH


We'll borrow the **get_category_type** function from the Foursquare lab.

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<a id='item2'></a>

## Explore Neighborhoods in Manhattan

#### Let's create a function to get near by venues for all the neighborhoods in Manhattan

In [13]:
radius= 500
LIMIT= 100

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [15]:
# just checking
manhattan_data.shape

(40, 4)

In [16]:
# getNearbyVenues(names, latitudes, longitudes, radius=500)
manhattan_venues= getNearbyVenues(manhattan_data['Neighborhood'], manhattan_data['Latitude'], manhattan_data['Longitude'])


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [17]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop


In [29]:
# just for back-up 
manhattan_venues.to_csv('manhattan_venues.csv')

#### Let's check the size of the resulting dataframe

In [18]:
manhattan_venues.shape

(3297, 7)

#### Let's find out how many unique categories are there from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 322 uniques categories.


In [20]:
manhattan_venues['Venue Category'].unique()

array(['Pizza Place', 'Yoga Studio', 'Diner', 'Coffee Shop', 'Donut Shop',
       'Seafood Restaurant', 'Department Store', 'Tennis Stadium', 'Gym',
       'Pharmacy', 'Video Game Store', 'Discount Store',
       'Supplement Shop', 'Shoe Store', 'Ice Cream Shop',
       'Comfort Food Restaurant', 'Bank', 'Steakhouse', 'Sandwich Place',
       'Deli / Bodega', 'Chinese Restaurant', 'General Entertainment',
       'Cocktail Bar', 'Greek Restaurant', 'English Restaurant', 'Museum',
       'Noodle House', 'Hotel', 'Garden Center', 'Spa',
       'American Restaurant', 'Bubble Tea Shop', 'Roof Deck', 'Tea Room',
       'New American Restaurant', 'Bar', 'Indie Movie Theater',
       'Malay Restaurant', 'Salon / Barbershop', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Hotpot Restaurant', 'Bike Shop',
       'Dim Sum Restaurant', 'Italian Restaurant', 'Jewelry Store',
       'Spanish Restaurant', 'Music Venue', 'Historic Site',
       'Asian Restaurant', 'Organic Grocery', 'Nail 

In [21]:
type(manhattan_venues['Venue Category'].unique())

numpy.ndarray

## Lets try to understand the different restaurant categories available

In [22]:
venue_cat_df = pd.DataFrame(manhattan_venues['Venue Category'].unique())

In [23]:
venue_cat_df.head()

,0
0,Pizza Place
1,Yoga Studio
2,Diner
3,Coffee Shop
4,Donut Shop


In [24]:
venue_cat_df[venue_cat_df[0].str.contains("Restaurant")]

,0
5,Seafood Restaurant
15,Comfort Food Restaurant
20,Chinese Restaurant
23,Greek Restaurant
24,English Restaurant
30,American Restaurant
34,New American Restaurant
37,Malay Restaurant
40,Vietnamese Restaurant
41,Hotpot Restaurant


### By looking at the above list, the following will come under - 'Chinese (or similar)':

### Chinese Restaurant, Dim Sum Restaurant, Dumpling Restaurant, Shanghai Restaurant, Szechuan Restaurant

In [25]:
# Note: this is based on our understanding of food similar to Chinese
# This can be changed if needed

<a id='item3'></a>

## Analyze Each Neighborhood

In [26]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center

In [27]:
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spi

And let's examine the new dataframe size.

In [28]:
manhattan_onehot.shape

(3297, 323)

#### Next, let's group rows by neighborhood and by taking the **SUM** of the frequency of occurrence of each category

In [30]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').sum().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spi

### Now lets check for our target categories

In [31]:
manhattan_grouped_2 = manhattan_grouped[['Neighborhood','Chinese Restaurant', 'Dim Sum Restaurant', 'Dumpling Restaurant', 'Shanghai Restaurant', 'Szechuan Restaurant']]

In [32]:
manhattan_grouped_2.head()

,Neighborhood,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Shanghai Restaurant,Szechuan Restaurant
0,Battery Park City,1,0,0,0,0
1,Carnegie Hill,1,0,0,0,0
2,Central Harlem,2,0,0,0,0
3,Chelsea,0,0,0,0,0
4,Chinatown,9,4,3,1,0


In [33]:
manhattan_grouped_2.dtypes

Neighborhood           object
Chinese Restaurant      uint8
Dim Sum Restaurant      uint8
Dumpling Restaurant     uint8
Shanghai Restaurant     uint8
Szechuan Restaurant     uint8
dtype: object

In [34]:
Total = []

In [35]:
for row in range(0,manhattan_grouped_2.shape[0]):
    t= (manhattan_grouped_2.loc[row,'Chinese Restaurant':].values).astype('float').sum()
    Total.append(t)

In [36]:
manhattan_grouped_2['Total'] = pd.DataFrame(Total)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
manhattan_grouped_2

,Neighborhood,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Shanghai Restaurant,Szechuan Restaurant,Total
0,Battery Park City,1,0,0,0,0,1.0
1,Carnegie Hill,1,0,0,0,0,1.0
2,Central Harlem,2,0,0,0,0,2.0
3,Chelsea,0,0,0,0,0,0.0
4,Chinatown,9,4,3,1,0,17.0
5,Civic Center,0,0,0,0,0,0.0
6,Clinton,1,1,0,0,0,2.0
7,East Harlem,1,0,0,0,0,1.0
8,East Village,3,0,1,0,0,4.0
9,Financial District,1,0,0,0,0,1.0


### Lets visualize the number of restaurants over the map of Manhattan

In [38]:
manhattan_data.head(2)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279


In [39]:
manhattan_grouped_2 = manhattan_grouped_2.merge(manhattan_data, on='Neighborhood')

In [40]:
manhattan_grouped_2.head()

,Neighborhood,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Shanghai Restaurant,Szechuan Restaurant,Total,Borough,Latitude,Longitude
0,Battery Park City,1,0,0,0,0,1.0,Manhattan,40.711932,-74.016869
1,Carnegie Hill,1,0,0,0,0,1.0,Manhattan,40.782683,-73.953256
2,Central Harlem,2,0,0,0,0,2.0,Manhattan,40.815976,-73.943211
3,Chelsea,0,0,0,0,0,0.0,Manhattan,40.744035,-74.003116
4,Chinatown,9,4,3,1,0,17.0,Manhattan,40.715618,-73.994279


In [41]:
manhattan_grouped_2.sort_values(by= ['Total'], axis=0, ascending=False, inplace=True)

In [42]:
manhattan_grouped_2

,Neighborhood,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Shanghai Restaurant,Szechuan Restaurant,Total,Borough,Latitude,Longitude
4,Chinatown,9,4,3,1,0,17.0,Manhattan,40.715618,-73.994279
18,Little Italy,4,0,0,0,0,4.0,Manhattan,40.719324,-73.997305
8,East Village,3,0,1,0,0,4.0,Manhattan,40.727847,-73.982226
19,Lower East Side,3,0,0,0,0,3.0,Manhattan,40.717807,-73.980890
23,Midtown,2,0,0,0,1,3.0,Manhattan,40.754691,-73.981669
12,Greenwich Village,3,0,0,0,0,3.0,Manhattan,40.726933,-73.999914
31,Sutton Place,1,0,0,0,1,2.0,Manhattan,40.760280,-73.963556
15,Inwood,2,0,0,0,0,2.0,Manhattan,40.867684,-73.921210
26,Murray Hill,1,0,0,1,0,2.0,Manhattan,40.748303,-73.978332
13,Hamilton Heights,2,0,0,0,0,2.0,Manhattan,40.823604,-73.949688


In [43]:
top_10 = manhattan_grouped_2.head(10)

In [44]:
top_10.head(2)

,Neighborhood,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Shanghai Restaurant,Szechuan Restaurant,Total,Borough,Latitude,Longitude
4,Chinatown,9,4,3,1,0,17.0,Manhattan,40.715618,-73.994279
18,Little Italy,4,0,0,0,0,4.0,Manhattan,40.719324,-73.997305


In [45]:
# create map of Manhattan using latitude and longitude values
geolocator = Nominatim()
location = geolocator.geocode('Manhattan, NY')
latitude = location.latitude
longitude = location.longitude

map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, poi, totno in zip(top_10['Latitude'], top_10['Longitude'], top_10['Neighborhood'], top_10['Total']):
    label = folium.Popup(str(poi) + ' Total No ' + str(totno), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


## The above map shows the top 10 neighborhoods in Manhattan with respect to number of Chinese/ similar restaurants.Based on the data our target audience can take further actions 

### Discussion
#### Some limitation of the models are:
- here we have not done any sizing of the potential customer base, i.e. number of people per neighborhood 
who may be interested in Chinese food. This can subsequently be done as further refinement

## End of Notebook